In [ ]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [17]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)
    layer = layers.add([layer,
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
        layers.Softmax(name='K')(layers.Dense(KINDS_SIZE)(layers.LSTM(64)(layer)))
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = keras.Model(inputs=[input_text], outputs=outputs[:-1])
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 64, 110)      4840        input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 64, 220)      582560      embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 64, 220)      776160      bidirectional_4[0][0]            
____________________________________________________________________________________________

In [3]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    #     train.kind = np.ones((len(train), 1))
    #     valid.kind = np.ones((len(valid), 1))
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized, 'K': train.kind }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized, 'K': valid.kind }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [ ]:
data_rabanit = load_data(['rabanit'])

In [ ]:
data_pre_modern = load_data(['pre_modern'])

In [5]:
data_modern = load_data(validation=0.2, source=['modern'])

(13856,)
(13856, 64)


In [8]:
data_all = load_data(validation=0.1, source=['biblical', 'rabanit', 'pre_modern', 'modern'])

(154697,)
(154697, 64)


In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

In [ ]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

In [6]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.5e-3))
# history = fit(data_modern, CircularLearningRate(1e-3, 2e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

Train on 13856 samples, validate on 3464 samples
13856/13856 [==============================] - 29s 2ms/sample - loss: 1.1777 - N_loss: 0.5031 - D_loss: 0.1207 - S_loss: 0.0247 - K_loss: 0.5293 - N_accuracy: 0.8198 - D_accuracy: 0.9525 - S_accuracy: 0.9941 - K_accuracy: 0.9978 - val_loss: 0.8426 - val_N_loss: 0.2496 - val_D_loss: 0.0695 - val_S_loss: 0.0068 - val_K_loss: 0.5169 - val_N_accuracy: 0.9101 - val_D_accuracy: 0.9720 - val_S_accuracy: 0.9981 - val_K_accuracy: 1.0000


In [23]:
model.load_weights('./checkpoints/uninit')
history = fit(data_all, scheduler=CircularLearningRate(20e-4, 80e-4, 5e-4))
# history = fit(data_modern, CircularLearningRate(1e-3, 2e-3, 0.5e-3))
model.save_weights('./checkpoints/all')

Train on 154697 samples, validate on 17189 samples
154697/154697 [==============================] - 220s 1ms/sample - loss: 1.0118 - N_loss: 0.4164 - D_loss: 0.1006 - S_loss: 0.0179 - K_loss: 0.4768 - N_accuracy: 0.8534 - D_accuracy: 0.9610 - S_accuracy: 0.9959 - K_accuracy: 0.7872 - val_loss: 0.4476 - val_N_loss: 0.2008 - val_D_loss: 0.0576 - val_S_loss: 0.0052 - val_K_loss: 0.1839 - val_N_accuracy: 0.9294 - val_D_accuracy: 0.9774 - val_S_accuracy: 0.9986 - val_K_accuracy: 0.9333


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [ ]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [20]:
model.load_weights('./checkpoints/all')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin, actual_kind] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2]),  dataset.from_categorical(prediction[3])]
    [expected_niqqud, expected_dagesh, expected_sin, expected_kind] = [data.niqqud[s], data.dagesh[s], data.sin[s], data.kind[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a, actual_kind[i])
        print('רצוי: ', e, expected_kind[i])

print_predictions(data_all[1], 5)

מצוי:  תּוֹהוֹת בְּאֶפֶס תִּקְוָה, עַל-כָּל-פָּרָשַׁת דְּרָכִים וְלְיַד כָּל-רֹאשׁ נְתִיבוֹת: הַיָּבֹא  3
רצוי:  תּוֹהוֹת בְּאֶפֶס תִּקְוָה, עַל-כָּל-פָּרָשַׁת דְּרָכִים וּלְיַד כָּל-רֹאשׁ נְתִיבוֹת: הֲיָבֹא  3
מצוי:  הִזְחִילוּ אֶל מִתַּחַת לְרִצְפָּה, וְדָגִים שׁוּחִים בְּתוֹךְ הַמַּיִם. וַיָּבֹא שְׁלֹמֹה וַיָּשַׁב עַל  3
רצוי:  הִזְחִילוּ אֶל מִתַּחַת לָרִצְפָּה, וְדָגִים שׂוֹחִים בְּתֹוךְ הַמַּיִם. וַיָּבֹא שְׁלֹמֹה וַיֵּשֶׁב עַל  3
מצוי:  יִשְׂרְאֵלִית לֹא תִּתְיַחַד עִם גּוֹי אֲפִילוּ אִשְׁתּוֹ עִמּוֹ וַאֲפִלּוּ הֵם הַרְבֵּה גּוִיִם  1
רצוי:  יִשְׂרָאֵלית לֹא תִתְיַחֵד עִם גּוֹי אֲפִילוּ אִשְׁתּוֹ עִמוֹ וַאֲפִלּוּ הֵם הַרְבֵּה גוֹיִם  1
מצוי:  וּכְעָבֹר רֶגַע כְּבָר יָשַׁבְנוּ עַל הַחוֹל וּלְעַסְנוּ בִּיסְקוִיט וּבָשָׂר מִמֶּלָּח. הַשּׁוֹדְדִים  4
רצוי:  וְכַעֲבֹר רֶגַע כְּבָר יָשַׁבְנוּ עַל הַחוֹל וְלָעַסְנוּ בִּיסְקְוִיט וּבָשָׂר מְמֻלָח. הַשׁוֹדְדִים  3
מצוי:  מְיֻחָד לָאָדָם בִּמָקוֹם אַחֵר--בִּמְקוֹם שְׁהוּא מְיֻחָד בּוֹ, אֵינוּ צָרִיךְ מַחְשָׁבָה לְיַחֲדוֹ  1
רצוי:  מְיֻחָד

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])